In [ ]:
# The following must be done before this step:

# Run ROJ macro in TerraScan to get nVeg, nTot, hMax for
# the blocks along the powerlines

# Run the FME script
# Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2020\fme\Beräkna_HF_TF\...
# wsr_compute_HF_TF.fmw
# which computes vegetation ratio, reclassifies height and vegetation
# ratio to HF and TF, merges blocks per line

# Output from FME, and input to the cell below, is
# LGXXX\line_Y\ROJ\...
# ...HF_1p5.tif and 
# ...TF_1p5.tif

In [1]:
import arcpy
import SVK
import os
import shutil
import pandas as pd

arcpy.CheckOutExtension("Spatial")

'CheckedOut'

# Calculate SF

In [31]:
# Computes SF_4p5.tif 
# by first computing HF_4p5 and TF_4p5, and then their sum

LGs_info_path = r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ledningar_2021_sample.txt"
workdir = r"\\sweco.se\SE\LLA01\PROJEKT\26664\6605189\000\Arbetsmapp_analys\SVK_2021"

# calculate processing extent
def calc_processing_extent(rawRaster, outputCellSize):
    rasterObj = arcpy.sa.Raster(rawRaster)
    inputExtent = rasterObj.extent
    # output xmin, ymin, xmax, ymax must be a multiple of output cell size
    # output xmin, ymin must be equal to or smaller than input xmin, ymin
    # output xmax, ymax must be equal to or greater than input xmax, ymax    
    XMinOut = inputExtent.XMin - inputExtent.XMin%outputCellSize
    YMinOut = inputExtent.YMin - inputExtent.YMin%outputCellSize
    XMaxOut = inputExtent.XMax - inputExtent.XMax%outputCellSize + outputCellSize
    YMaxOut = inputExtent.YMax - inputExtent.YMax%outputCellSize + outputCellSize
    processingExtent = f"{XMinOut} {YMinOut} {XMaxOut} {YMaxOut}"
    return(processingExtent)


# aggregate from 1.5 to 4.5 m cells, using median
def aggr(inRaster, outRaster, processingExtent):
    # Set processing extent for aggregate and clipping operation
    arcpy.env.extent = processingExtent
    aggrRaster = arcpy.sa.Aggregate(inRaster, 3, "MEDIAN", "#", "#")
    aggrRaster.save(outRaster)
    print("Aggregation done")

    
# add HF and TF to SF
def compute_SF(HF, TF, outSF):
    SF = arcpy.Raster(HF) + arcpy.Raster(TF)
    SF.save(outSF)


def HF_TF_to_SF(powerline):
    
    LG = powerline["LG"]
    line = powerline["line"]

    line_name = f"line_{line}"
    print(f"Processing {LG}_{line}")

    line_dir = os.path.join(workdir, LG, line_name, "ROJ")
    HF_1p5 = os.path.join(line_dir, "HF_1p5.tif")
    TF_1p5 = os.path.join(line_dir, "TF_1p5.tif")
    HF_4p5 = os.path.join(line_dir, "HF_4p5.tif")
    TF_4p5 = os.path.join(line_dir, "TF_4p5.tif")
    SF = os.path.join(line_dir, "SF_4p5.tif")

    processing_extent = calc_processing_extent(HF_1p5, 4.5)
    
    # aggregate 3*3 (median)
    print("\tAggregating HF")
    aggr(HF_1p5, HF_4p5, processing_extent)
    print("\tAggregating TF")
    aggr(TF_1p5, TF_4p5, processing_extent)
        
    # Calculate SF as HF + TF, and save
    print("\tComputing SF")
    compute_SF(HF_4p5, TF_4p5, SF)
    print(f"\tDone with {LG}, {line_name}")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(HF_TF_to_SF, axis=1)

Processing LG710_2
	Aggregating HF
Aggregation done
	Aggregating TF
Aggregation done
	Computing SF
	Done with LG710, line_2
Processing LG730_1
	Aggregating HF
Aggregation done
	Aggregating TF
Aggregation done
	Computing SF
	Done with LG730, line_1


0    None
1    None
dtype: object

# Merge all lines in LG
## for SF, HF, TF respectively

In [28]:
##### SPECALLÖSNING FÖR LG044_1 SOM ÄR SÅ LÅNG ATT DEN BEHÖVDE DELAS UPP FÖR ATT KUNNA KÖRAS
#
## SLÅ IHOP DELARNA AV HF_1p5, TF_1p5, SF_4p5 INNAN STEGET NEDAN
#workdir = r"\\sweco.se\SE\LLA01\PROJEKT\26664\6605189\000\Arbetsmapp_analys\SVK_2021\LG044\line_1\ROJ"
#
#all_rasters = {"HF_1p5":["HF_1p5_1-60.tif", "HF_1p5_61-120.tif", "HF_1p5_121-162.tif"],
#               "TF_1p5":["TF_1p5_1-60.tif", "TF_1p5_61-120.tif", "TF_1p5_121-162.tif"],
#               "SF_4p5":["SF_4p5_1-60.tif", "SF_4p5_61-120.tif", "SF_4p5_121-162.tif"]}
#
## HF_raster_names = ["HF_1p5_1-60.tif", "HF_1p5_61-120.tif", "HF_1p5_121-162.tif"]
## TF_raster_names = ["TF_1p5_1-60.tif", "TF_1p5_61-120.tif", "TF_1p5_121-162.tif"]
## SF_raster_names = ["SF_4p5_1-60.tif", "SF_4p5_61-120.tif", "SF_4p5_121-162.tif"]
#
#arcpy.env.workspace = workdir
#
#for raster_type in all_rasters:
#    raster_names_ = all_rasters[raster_type]
#    raster_names = ";".join(raster_names_)
#    merged_raster_name = raster_type + ".tif"
#    
#   # print(raster_names)
#   # print(merged_raster_name, "\n")
#    arcpy.management.MosaicToNewRaster(raster_names, workdir, merged_raster_name, "#", "8_BIT_UNSIGNED", None,  1, "MAXIMUM", "#")
##    arcpy.management.MosaicToNewRaster(raster_names, workdir, merged_raster_name, "#", "#", None, 1, "MAXIMUM", "#")                                       )




In [47]:
workdir = r"\\sweco.se\SE\LLA01\PROJEKT\26664\6605189\000\Arbetsmapp_analys\SVK_2021"
LGs_info_path = r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ledningar_2021_sample.txt"
raster_types = ["HF_1p5", "TF_1p5", "SF_4p5"]

#def copy_rasters(LG, line, rasterTypes):
#    #LG_dir = os.path.join(workdir, LG) (GLOBAL, SO ALREADY DEFINED)
#    inputDir = os.path.join(LG_dir, f"line_{line}", "ROJ")
#    #output_dir = os.path.join(LG_dir, "ROJ_merged") (GLOBAL, SO ALREADY DEFINED)
#    
#    for rasterType in rasterTypes:
#        print(f"\tCopying {rasterType}")
#        inputRaster = os.path.join(inputDir, f"{rasterType}.tif")
#        outputRaster = os.path.join(output_dir, f"{rasterType}.tif")
#        shutil.copyfile(inputRaster, outputRaster)

        
def get_input_rasters(LG, lines, rasterType):
    # LG_dir = os.path.join(workdir, LG_name) (GLOBAL, SO ALREADY DEFINED)
    
    inputRasters_ = []
    print(f"\tMaking a list of all {rasterType} rasters in {LG}")
    for line in lines:
        inputRaster = os.path.join(LG_dir, f"line_{line}", "ROJ", f"{rasterType}.tif")
        inputRasters_.append(inputRaster)
    
    inputRasters = ";".join(inputRasters_)
    
    return(inputRasters)
        

def merge_rasters(LG, lines, rasterTypes):    
    for rasterType in rasterTypes:
        # List raster (SF or HF or TF) from all lines in LG
        inputRasters = get_input_rasters(LG, lines, rasterType)
        outputRaster = f"{rasterType}.tif"
        print(f"\tMerging rasters and saving as {outputRaster}")
        arcpy.MosaicToNewRaster_management(inputRasters, 
                                           output_dir, 
                                           outputRaster, 
                                           "#", 
                                           "#", 
                                           None, 
                                           1, 
                                           "MAXIMUM", 
                                           "#")

        
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs = LGs_info.LG.unique()

# För varje LG
for LG in LGs:
    print(f"Processing {LG}")
    LG_dir = os.path.join(workdir, LG)
    output_dir = os.path.join(LG_dir, "ROJ_merged")
    lines = LGs_info[LGs_info["LG"] == LG]["line"]

#    Verkade bli nåt fel med kopiering (områden utanför intrångsersatt blev 0, inte nodata(255?), eller så hände det i
#    ett senare steg, men för säkerhets och tydlighets skull gör jag merge på alla LG även de som bara har 1 ledning)
#    SVK.create_dir(output_dir)
#    # If LG only consists of one line, copy HF_1p5, TF_1p5, SF_4p5 instead of merge
#    if len(lines) == 1:
#        print(f"\tOnly one line in LG, copy rasters without merging")
#        line = lines[0]
#        copy_rasters(LG, line, raster_types)
#        print(f"\tDone copying ROJ rasters")

#    # Else, if LG consists of more than one line, 
#    # merge HF, TF and SF rasters for all lines into one
#    # Merge to one SF or HF or TF raster for the whole LG
#    else:
#        print(f"\tMerging rasters from {len(lines)} lines")
#        merge_rasters(LG, lines, raster_types)

    print(f"\tMerging rasters from {len(lines)} lines")
    merge_rasters(LG, lines, raster_types)


Processing LG044
	Merging rasters from 5 lines
	Making a list of all HF_1p5 rasters in LG044
	Merging rasters and saving as HF_1p5.tif
	Making a list of all TF_1p5 rasters in LG044
	Merging rasters and saving as TF_1p5.tif
	Making a list of all SF_4p5 rasters in LG044
	Merging rasters and saving as SF_4p5.tif


In [45]:
import json
import codecs

employees_path = r"C:\Users\SEKNNO\Desktop\PeterPythonTMP\Employees.json"

list_emails = []
employees_content = codecs.open(employees_path, "r", encoding="latin-1")
employees = json.load(employees_content)["data"]["employees"]["results"]

for employee in employees:
    email = employee["email"] 
    if email is not None:
        list_emails.append(email)
        
print(list_emails)

['aadharsh.sagar@swecogroup.in', 'abhishek.rawat@swecogroup.in', 'abhilash.joshi@swecogroup.in', 'ajay.yadav@swecogroup.in', 'akhilesh.kumar@swecogroup.in', 'amanshardawadh.mewara@swecogroup.in', 'amitsingh.rawat@swecogroup.in', 'anant.kesarwani@swecogroup.in', 'ankit.majawadia@swecogroup.in', 'anil.kumar@swecogroup.in', 'ankit.singh2@swecogroup.in', 'anuradha.singh@swecogroup.in', 'arpit.shukla@swecogroup.in', 'ashish.devgan@swecogroup.in', 'atul.kashyap@swecogroup.in', 'aviral.shukla@swecogroup.in', 'bhaskar.kaushik@swecogroup.in', 'bheru.patidar@swecogroup.in', 'bhawna.sharma@swecogroup.in', 'biswajit.sahoo@swecogroup.in', 'deepshikha.sharma@swecogroup.in', 'deepmala.upadhyay@swecogroup.in', 'deepak.sharma@swecogroup.in', 'dheeraj.kumar@swecogroup.in', 'dushyant.sharma@swecogroup.in', 'gourhari.das@swecogroup.in', 'gauravsingh.jaggi@swecogroup.in', 'gaurav.tiwari@swecogroup.in', 'm.gupta@swecogroup.in', 'himanshu.sharma@swecogroup.in', 'hussain.thaha@swecogroup.in', 'irshad.wahid@sw

In [46]:
import json
import codecs

listEmails = []

content = codecs.open(employees_path, "r", encoding="latin-1")
data = json.load(content)["data"]["employees"]["results"]

for i in data:
    if i["email"] is not None:
        listEmails.append(i["email"])

print(listEmails)

['aadharsh.sagar@swecogroup.in', 'abhishek.rawat@swecogroup.in', 'abhilash.joshi@swecogroup.in', 'ajay.yadav@swecogroup.in', 'akhilesh.kumar@swecogroup.in', 'amanshardawadh.mewara@swecogroup.in', 'amitsingh.rawat@swecogroup.in', 'anant.kesarwani@swecogroup.in', 'ankit.majawadia@swecogroup.in', 'anil.kumar@swecogroup.in', 'ankit.singh2@swecogroup.in', 'anuradha.singh@swecogroup.in', 'arpit.shukla@swecogroup.in', 'ashish.devgan@swecogroup.in', 'atul.kashyap@swecogroup.in', 'aviral.shukla@swecogroup.in', 'bhaskar.kaushik@swecogroup.in', 'bheru.patidar@swecogroup.in', 'bhawna.sharma@swecogroup.in', 'biswajit.sahoo@swecogroup.in', 'deepshikha.sharma@swecogroup.in', 'deepmala.upadhyay@swecogroup.in', 'deepak.sharma@swecogroup.in', 'dheeraj.kumar@swecogroup.in', 'dushyant.sharma@swecogroup.in', 'gourhari.das@swecogroup.in', 'gauravsingh.jaggi@swecogroup.in', 'gaurav.tiwari@swecogroup.in', 'm.gupta@swecogroup.in', 'himanshu.sharma@swecogroup.in', 'hussain.thaha@swecogroup.in', 'irshad.wahid@sw

# Krångel med tiles i LG044 (lång ledning)

In [7]:
# LG044 så lång att det inte går att mergea allt hur som helst. Delar upp line_1 i tre delar: block 1-60, 61-120, 121-162. 
# Gör tiles av varje del för sig, en mapp per del. Lines 2-5 görs tillsamans i en egen del (mapp)
# Detta skript gör först en lista över alla tiles som finns (samtliga mappar) och går sedan igenom dem en och en:
# Om en tile finns i endaast en mapp så kopieras den till slutmappen.
# Om en tile finns i flera mappar slås dessa ihop (maximum) och sparas i slutmappen.

from pathlib import Path

prj_dir = Path(r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ROJ_tiles\LG044\TF_1p5")
linedirs_tiles = {"line_1_1-60":[], "line_1_61-120":[], "line_1_121-162":[], "line_2-5":[]}

os.chdir(prj_dir)

def merge_tiles(tiles, tilename):
    arcpy.MosaicToNewRaster_management(tiles, str(prj_dir), tilename, "#", "#", None, 1, "MAXIMUM", "#")
    
for line_dir in linedirs_tiles:
    tiles = Path(line_dir).glob("R*.tif")
    tilenames = [tile.name for tile in tiles]
    print(len(tilenames))
    linedirs_tiles[line_dir] = tilenames

unique_tiles = list(set(linedirs_tiles["line_1_1-60"]+
                        linedirs_tiles["line_1_61-120"]+
                        linedirs_tiles["line_1_121-162"]+
                        linedirs_tiles["line_2-5"]))

for tile in unique_tiles:
    paths_for_tile_ = prj_dir.glob("*\\"+tile)
    paths_for_tile = [str(p) for p in paths_for_tile_]
        
    if len(paths_for_tile) == 1:
        shutil.copy(paths_for_tile[0], prj_dir / tile)
        print(f"Copy {tile}")
    else:
        merge_tiles(paths_for_tile, tile)
        print(f"Merge {tile}")
        

        
#        my_file=pathlib.Path('/etc/hosts')
#to_file=pathlib.Path('/tmp/foo')
#shutil.copy(my_file, to_file)
#print(len(line_tiles["line_1_1-60"])+
#      len(line_tiles["line_1_61-120"])+
#      len(line_tiles["line_1_121-162"])+
#      len(line_tiles["line_2-5"]))
#print(len(unique_tiles))

291
297
185
434
Copy R5595_67530.tif
Merge R5260_68905.tif
Merge R5490_68145.tif
Merge R5505_68130.tif
Copy R5610_67770.tif
Merge R5515_68100.tif
Copy R5275_69570.tif
Merge R5540_68050.tif
Copy R5245_69640.tif
Copy R5605_67720.tif
Copy R5425_67065.tif
Copy R5335_69400.tif
Copy R5475_67230.tif
Merge R5590_67920.tif
Merge R5315_69105.tif
Merge R5345_68445.tif
Merge R5365_68405.tif
Merge R5305_69075.tif
Merge R5430_68285.tif
Merge R5480_68170.tif
Merge R5260_68780.tif
Merge R5535_68060.tif
Copy R5605_67845.tif
Merge R5565_67970.tif
Merge R5300_68575.tif
Copy R5545_67415.tif
Copy R5535_67390.tif
Merge R5485_68170.tif
Merge R5350_69240.tif
Copy R5535_67375.tif
Copy R5490_67265.tif
Merge R5595_67905.tif
Merge R5305_68570.tif
Merge R5260_68770.tif
Merge R5480_68180.tif
Copy R5540_67395.tif
Copy R5495_67265.tif
Copy R5310_69475.tif
Merge R5305_68565.tif
Merge R5260_68785.tif
Merge R5350_69225.tif
Copy R5250_69620.tif
Merge R5260_68805.tif
Merge R5395_68340.tif
Copy R5580_67490.tif
Merge R5325_

Copy R5530_67370.tif
Copy R5495_67275.tif
Merge R5350_68435.tif
Merge R5255_68820.tif
Copy R5350_69345.tif
Merge R5255_68855.tif
Merge R5255_68890.tif
Merge R5330_69175.tif
Merge R5490_68160.tif
Copy R5290_69535.tif
Copy R5600_67625.tif
Copy R5475_67225.tif
Copy R5605_67610.tif
Merge R5445_68265.tif
Merge R5460_68215.tif
Merge R5260_68750.tif
Merge R5495_68145.tif
Merge R5370_68390.tif
Copy R5600_67705.tif
Copy R5535_67385.tif
Copy R5540_67410.tif
Copy R5600_67685.tif
Merge R5260_68790.tif
Copy R5290_69550.tif
Merge R5475_68185.tif
Merge R5360_68420.tif
Copy R5455_67185.tif
Copy R5605_67575.tif
Merge R5275_68945.tif
Copy R5460_67200.tif
Merge R5265_68730.tif
Copy R5340_69390.tif
Merge R5350_69235.tif
Copy R5350_69350.tif
Copy R5600_67695.tif
Merge R5325_69155.tif
Merge R5285_68605.tif
Merge R5355_68430.tif
Copy R5490_67270.tif
Copy R5355_69345.tif
Copy R5285_69550.tif
Merge R5455_68230.tif
Copy R5480_67240.tif
Copy R5550_67445.tif
Copy R5450_67170.tif
Merge R5570_67960.tif
Copy R5610_6

Merge R5295_69030.tif
Merge R5350_69245.tif
Merge R5360_69310.tif
Merge R5345_69270.tif
Merge R5340_68460.tif
Copy R5330_69410.tif
Merge R5400_68340.tif
Merge R5385_68370.tif
Copy R5320_69450.tif


In [37]:
shutil.copy(Path(r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ROJ_tiles\LG044\line_1_1-60\R5350_69345.tif"), Path(r"Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ROJ_tiles\LG044\R5350_69345.tif"))

WindowsPath('Z:/26664/6605189/000/Arbetsmapp_analys/SVK_2021/ROJ_tiles/LG044/R5350_69345.tif')

In [50]:
tile = "R5315_68530.tif"
occurences = prj_dir.glob("*\\"+tile)

occ = [str(o) for o in occurences]

merge_tiles(occ, tile)
print(occ)
print(str(prj_dir))
print(tile)

['Z:\\26664\\6605189\\000\\Arbetsmapp_analys\\SVK_2021\\ROJ_tiles\\LG044\\line_1_61-120\\R5315_68530.tif', 'Z:\\26664\\6605189\\000\\Arbetsmapp_analys\\SVK_2021\\ROJ_tiles\\LG044\\line_2-5\\R5315_68530.tif']
Z:\26664\6605189\000\Arbetsmapp_analys\SVK_2021\ROJ_tiles\LG044
R5315_68530.tif


In [22]:
d = {"a":[1, 2, 3], "b":[4, 5, 6]}




print(aa)


[[[[1, 2, 3], [4, 5, 6], 1, 2, 3], [[1, 2, 3], [4, 5, 6], 4, 5, 6], 1, 2, 3], [[[1, 2, 3], [4, 5, 6], 1, 2, 3], [[1, 2, 3], [4, 5, 6], 4, 5, 6], 4, 5, 6]]
